In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import requests

list_of_files=requests.get('https://cortx.herokuapp.com/getFiles')
print(list_of_files.content)

b'[{"test.csv":"Tue, 27 Apr 2021 11:55:31 GMT","train.csv":"Tue, 27 Apr 2021 11:55:42 GMT"},200]\n'


In [3]:
import requests

response = requests.post('https://cortx.herokuapp.com/download',json={"key":"test.csv"})
open('test.csv', 'wb').write(response.content)

314666

In [4]:
import requests

response = requests.post('https://cortx.herokuapp.com/download',json={"key":"train.csv"})
open('train.csv', 'wb').write(response.content)

469156

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import torch.utils.data

In [6]:
trainDataset = pd.read_csv("train.csv", header=None)
testDataset = pd.read_csv("test.csv", header=None)

inputSize = len(trainDataset.columns) -1
hidden1Size = 128
hidden2Size = 128
numClasses = 2
numEpoch = 100
batchSize = 100
learningRate = 0.001

In [7]:
trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(trainDataset.values), batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=torch.tensor(testDataset.values), batch_size=batchSize, shuffle=False)

In [8]:

class DeepNeuralNetwork(nn.Module):
    def __init__(self, inputSize, hidden1Size, hidden2Size, numClasses):
        super(DeepNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(inputSize, hidden1Size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1Size, hidden2Size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden1Size, numClasses)
        self.logsm1 = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.logsm1(out)
        return out

In [9]:
dnn = DeepNeuralNetwork(inputSize, hidden1Size, hidden2Size, numClasses)

In [10]:
lossFN = nn.NLLLoss()
optimizer = torch.optim.Adam(dnn.parameters(), lr=learningRate)

In [11]:
for epoch in range(0, numEpoch):
    for i, data in enumerate(trainLoader,0):
        labels = Variable(data[:,-1])
        data = Variable(data[:,0:30].float())
        optimizer.zero_grad()
        outputs = dnn(data)
        loss = lossFN(outputs, labels.long())
        loss.backward()
        optimizer.step() 
                                        
    print('Epoch [%d/%d], Loss: %.4f'
        %(epoch+1, numEpoch, loss.item()))

Epoch [1/100], Loss: 0.1715
Epoch [2/100], Loss: 0.3463
Epoch [3/100], Loss: 0.2728
Epoch [4/100], Loss: 0.0429
Epoch [5/100], Loss: 0.1495
Epoch [6/100], Loss: 0.1283
Epoch [7/100], Loss: 0.1240
Epoch [8/100], Loss: 0.0590
Epoch [9/100], Loss: 0.0857
Epoch [10/100], Loss: 0.1107
Epoch [11/100], Loss: 0.0480
Epoch [12/100], Loss: 0.0422
Epoch [13/100], Loss: 0.0109
Epoch [14/100], Loss: 0.0101
Epoch [15/100], Loss: 0.2054
Epoch [16/100], Loss: 0.1209
Epoch [17/100], Loss: 0.0828
Epoch [18/100], Loss: 0.0154
Epoch [19/100], Loss: 0.0215
Epoch [20/100], Loss: 0.0820
Epoch [21/100], Loss: 0.0510
Epoch [22/100], Loss: 0.0646
Epoch [23/100], Loss: 0.1136
Epoch [24/100], Loss: 0.0016
Epoch [25/100], Loss: 0.3145
Epoch [26/100], Loss: 0.1865
Epoch [27/100], Loss: 0.0330
Epoch [28/100], Loss: 0.0387
Epoch [29/100], Loss: 0.0220
Epoch [30/100], Loss: 0.0212
Epoch [31/100], Loss: 0.0121
Epoch [32/100], Loss: 0.0055
Epoch [33/100], Loss: 0.0043
Epoch [34/100], Loss: 0.0025
Epoch [35/100], Loss: 0

In [12]:

correct = 0
total = 0
for data in testLoader:
    labels = Variable(data[:,-1])
    data = Variable(data[:,0:30].float()) 
    outputs = dnn(data)
    _, predicted = torch.max(outputs.data, 1)  
    total += labels.size(0)
    correct += (predicted == labels.long()).sum()
    
print('Accuracy of the network on the data: %d %%' % (100 * correct / total))

Accuracy of the network on the data: 95 %
